In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [96]:
path_img_thunder = "../data/Thunderstorm/thunder_00021.jpg"
path_img_rain = "../data/Rainy/7280_2020_12_29 15_21_27_91.jpg"

In [122]:
img_thunder = cv2.imread(path_img_thunder)

cv2.namedWindow("1", cv2.WINDOW_AUTOSIZE)
cv2.imshow("1", img_thunder)
k = cv2.waitKey(0)
cv2.destroyAllWindows()

In [153]:
img_rain = cv2.imread(path_img_rain)

cv2.namedWindow("1", cv2.WINDOW_AUTOSIZE)
cv2.imshow("1", img_rain)
k = cv2.waitKey(0)
cv2.destroyAllWindows()

In [155]:
img_rain_gray = cv2.cvtColor(img_rain, cv2.COLOR_BGR2GRAY)
img_rain_gray = cv2.GaussianBlur(img_rain_gray, (5, 5), 0)
img_rain_canny = cv2.Canny(img_rain_gray, 100, 140)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 13))
img_dilated = cv2.dilate(img_rain_canny, kernel)

# 轮廓
# contours, hierarchy = cv2.findContours(img_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# print("number of contours: %d" % len(contours))
# img_dilated = cv2.drawContours(img_dilated, contours, -1, (0, 255, 255), 2)

num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(img_dilated, connectivity=8, ltype=cv2.CV_32S)

hight, width = img_dilated.shape
print(num_labels)
best_x, best_y, best_w, best_h, best_area = 0, 0, 0, 0, 0
for idx, (x, y, w, h, area) in enumerate(stats):
    if w >= width / 3 and x != 0 and y != 0 :
        if y + h < best_y + best_h or best_y == 0:
            best_x, best_y, best_w, best_h, best_area = x, y, w, h, area
            max_area = best_w * best_h
            flag = 0
            area_set = set()
            print(best_x, best_y, best_w, best_h, best_area)
            cv2.rectangle(img_dilated, (x, y), (x + w, y + h), (255, 255, 255))
            for i in range(best_w):
                for j in range(best_h):
                    # if flag:
                    #     break
                    area_set.add(stats[labels[y + j, x + i]][-1])
                    if stats[labels[y + j, x + i]][-1] == best_area:
                        sky_line_num = labels[y + j, x + i]
                        flag = 1
            print(area_set)
            print(sky_line_num)
        # print(x, y, w, h)
        # s_x, s_y = x, y
        # cv2.rectangle(img_dilated, (x, y), (x + w, y + h), (255, 255, 255))
        # s_x, s_y = centroids[idx]
        # print(s_x, s_y)
        # cv2.circle(img_dilated, (int(s_y), int(s_x)), 200, (255, 255, 255))
        # sky_line_num = labels[int(s_y), int(s_x)]

# print(int(s_y), int(s_x))
# print(img_dilated.shape)
# cv2.circle(img_dilated, (int(s_y), int(s_x)), 200, (255, 255, 255), -1)
black_img = np.zeros((hight, width, 3), dtype=np.uint8)
# cv2.circle(black_img, (int(s_y), int(s_x)), 200, (0, 0, 255), -1)
threshold = [0] * width
low_x, low_y = 0, 0

for i in range(hight):
    for j in range(width):
        if labels[i, j] == sky_line_num:
            threshold[j] = max(threshold[j], i)
            if threshold[j] > low_y:
                low_x, low_y = j, threshold[j]
            black_img[i, j] = [0, 0, 255]

for i in range(width):
    if not (best_x <= i <= best_x + best_w):
        threshold[i] = best_y

for i in range(width):
    black_img[0:threshold[i], i] = (255, 255, 255)
    black_img[threshold[i]: hight, i] = (0, 0, 0)
cv2.imshow("1", img_dilated)
# print(stats)
cv2.imshow("2", img_rain)
k = cv2.waitKey(0)
cv2.destroyAllWindows()

29
881 108 925 92 19726
{14227, 1751148, 19726}
6


In [33]:
# 图片融合
hight, width = 224, 224
img_thunder = cv2.resize(img_thunder, (224, 224))
img_rain = cv2.resize(img_rain, (224, 224))

img_fusion = cv2.addWeighted(img_rain, 0.6, img_thunder, 0.4, 0)
cv2.imshow("1", img_fusion)
# cv2.imshow("1", img_thunder)
k = cv2.waitKey(0)
cv2.destroyAllWindows()

In [135]:

img_canny = cv2.Canny(img_thunder, 100, 150)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
img_dilated = cv2.dilate(img_canny, kernel)
# img_dilated = img_dilated[:350, :]
d_h, d_w = img_dilated.shape
img_dilated = cv2.resize(img_dilated, ((best_y + best_h), int((best_y + best_h) * d_w / d_h)))
# result = cv2.bitwise_and(img_thunder, img_thunder, mask=img_dilated)
# cv2.imshow('1', img_dilated)
# result = result[:350, :]
# r_h, r_w, _ = result.shape
# result = cv2.resize(result, ((best_y + best_h), int((best_y + best_h) * r_w / r_h)))
# thunder_fusion = cv2.bitwise_and
cv2.imshow('1', img_dilated)
k = cv2.waitKey()
cv2.destroyAllWindows()

In [149]:
# 原图片roi
target_h, target_w = img_dilated.shape
roi_x = max(0, low_x - target_w // 2)
roi_y = 0
roi_h = target_h
roi_w = target_w
print(roi_x, roi_y, roi_w, roi_h)
# black_img = cv2.cvtColor(black_img, cv2.COLOR_BGR2GRAY)
roi_mask = black_img[roi_y: roi_y + roi_h, roi_x: roi_x + roi_w]
roi = img_rain[roi_y: roi_y + roi_h, roi_x: roi_x + roi_w, :]
roi_mask = cv2.bitwise_not(roi_mask)
final_mask = cv2.bitwise_or(roi_mask, img_dilated)
roi_mask = cv2.bitwise_not(roi_mask)
final_mask = cv2.bitwise_and(final_mask, roi_mask)

test_thunder = img_thunder[:, :, :]
test_thunder = cv2.resize(test_thunder, ((best_y + best_h), int((best_y + best_h) * d_w / d_h)))
thunder = cv2.bitwise_and(test_thunder, test_thunder, mask=final_mask)

final_mask_inv = cv2.bitwise_not(final_mask)

back_ground = cv2.bitwise_and(roi, roi, mask=final_mask_inv)

fusion = cv2.add(thunder, back_ground)
result_thunder = img_rain[:, :, :]
result_thunder[roi_y: roi_y + roi_h, roi_x: roi_x + roi_w, :] = fusion
cv2.imshow('1', result_thunder)
k = cv2.waitKey()
cv2.destroyAllWindows()

1380 0 200 305


In [151]:
# 图像色调转移
from color_transfer import color_transfer

path_img_thunder_1 = "../data/Thunderstorm/thunder_00898.jpg"
img_thunder_1 = cv2.imread(path_img_thunder_1)
transfer = color_transfer(img_thunder_1, img_rain)

cv2.imshow('1', transfer)
k = cv2.waitKey()
cv2.destroyAllWindows()